In [1]:
import pandas as pd
import numpy as np
import script as sc
# import tensorflow as tf

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

NameError: name 'tf' is not defined

# Data Import

In [4]:
# data = pd.read_csv(r"C:\Users\yashm\Google Drive\Data Capstone_\Project Folder\PreWideData\updated_wide.csv")
data = pd.read_csv(sc.load_Data('AGG-Yash', 'updated_wide.csv'))

In [5]:
data = data.drop(['Unnamed: 0'], axis = 1)
data['codes'] = data.activity.astype('category').cat.codes

In [6]:
classes = len(data.activity.unique())

In [7]:
data.head()

,Accelerometer X,Accelerometer Y,Accelerometer Z,activity,type,codes
0,-0.605,0.617,0.070,active,H01-01,1
1,-0.621,0.602,0.023,active,H01-01,1
2,-0.605,0.590,-0.016,active,H01-01,1
3,-0.590,0.602,-0.043,active,H01-01,1
4,-0.594,0.648,-0.047,active,H01-01,1


In [65]:
temp = np.insert(np.asarray(data['Accelerometer X'][0:80]), 1, data['Accelerometer Y'][0:80])
temp = np.insert(temp, 1, np.asarray(data['Accelerometer Z'][0:80]))
temp.shape
x = []

# Data Reshape

In [83]:
x = []
y = []
for i in range(0, len(data), 80):
    if (data['activity'][i] != 'Private / Not Coded'):
        temp = np.insert(np.asarray(data['Accelerometer X'][i:i+80]), 1, data['Accelerometer Y'][i:i+80])
        temp = np.insert(temp, 1, np.asarray(data['Accelerometer Z'][i:i+80]))
        x.append(temp)
    #     x.append(np.asarray([np.asarray(data['Accelerometer X'][i:i+80]), np.asarray(data['Accelerometer Y'][i:i+80]), np.asarray(data['Accelerometer Z'][i:i+80])]))
        y.append(data.codes[i])
x = np.asarray(x) ## formated data shape is n [x, y, z] where n is each second and len(x) = 80

In [84]:
x.shape, len(y)

((115250, 240), 115250)

# Train and Test creation

In [85]:
train_x = x[0:int(0.7*len(x))]
train_y = y[0:int(0.7*len(x))]
test_x = x[int(0.7*len(x)):len(x)]
test_y = y[int(0.7*len(y)):len(y)]

train_x.shape , len(train_y), test_x.shape, len(test_y)

((80675, 240), 80675, (34575, 240), 34575)

# Hyper Parameter Tuning

In [86]:
## set parameters
learning_rate = 0.01
training_iteration = 30
batch_size = 100
display_step = 2

In [1]:
n_nodes_hl1 = 100
n_nodes_hl2 = 100
n_nodes_hl3 = 100

n_classes = classes

## trainX, trainY, testX, testY needed

x = tf.placeholder('float', [None, 240]) ## 80 + 80 + 80
y = tf.placeholder('float', [None, n_classes])

NameError: name 'classes' is not defined

# CNN network functions

In [71]:
def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([240, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output


In [72]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        # OLD:
        #sess.run(tf.initialize_all_variables())
        # NEW:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            i = 0
            while i < len(train_x):
                start = i
                end = i + batch_size
                epoch_x = np.array(train_x[start:end])
                epoch_y = np.array(train_y[start:end])
            
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
                i += batch_size

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))

# Train CNN models

In [88]:
train_neural_network(x)

InvalidArgumentError: logits and labels must be broadcastable: logits_size=[100,7] labels_size=[1,100]
	 [[Node: softmax_cross_entropy_with_logits_sg_5 = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](softmax_cross_entropy_with_logits_sg_5/Reshape, softmax_cross_entropy_with_logits_sg_5/Reshape_1)]]

Caused by op 'softmax_cross_entropy_with_logits_sg_5', defined at:
  File "c:\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\python36\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "c:\python36\lib\site-packages\tornado\platform\asyncio.p